# **🚌 Bus Depot Problem**

**🧠 Algorithm:** Vogel's Approximation Method with a Total Cost Matrix

### **⚙️ Process**

1. **Initialization**: 
   * Set up supply (buses), demand (depots), and the cost matrix. 🚌 
2. **Penalty Calculation**:  
   * For each row/column, find the difference between the two lowest costs. This shows potential savings! 📉
3. **Allocation**: 
   * Pick the row/column with the biggest penalty.
   * Assign as much as you can to the cheapest spot in that row/column.
   * Update supply and demand.  ✅
4. **Iteration**: Repeat steps 2 and 3 until everything is assigned. 🔁 
5. **Total Cost Calculation**: Add up the costs of all the assignments. 💰

### **📦 Imported Packages**

* **pandas**: To work with data tables (DataFrames) 📊
* **numpy**: For number crunching  and arrays 🔢
* **random**: To add some randomness  to the simulation🎲
* **rich** (Maybe): For making the output look fancier ✨

### **📝 Variable Description**

* **no_of_depots (int)**: How many depots we have  🏭
* **depot_capacities (list)**: How much space each depot has 📦
* **df_demand (dictionary)**: How many buses each depot needs (key: depot name, value: demand)   {'D1': 20, 'D2': 35, ...} 🚌 
* **total_demand (int)**: Total buses needed across all depots 🚌🚌🚌...
* **df_supply (dictionary)**: Available buses (usually starts with 1 bus per entry) 🚌
* **total_supply (int)**:  Total number of available buses 🚌🚌🚌...
* **df_dkm (dataframe)**: Distances or costs to move an empty bus between locations 🛣️ 
* **vehicle_ages (list)**: How old each bus is 🚌👴
* **age (dictionary)**:  Maps each bus to its age 
* **kpl (list)**: Kilometers per liter (fuel efficiency) for each bus, based on age ⛽️
* **co2_emissions (list)**: CO2 emissions for each bus, based on age 💨
* **fc_kpl (dictionary)**: Fuel cost per kilometer for each bus 💸
* **co2_c (dictionary)**: Cost related to CO2 emissions for each bus 💸
* **df_doc (list)**: Depot operating costs  💼
* **df_tdkom (dataframe)**: The big one! Total cost to park a bus at each depot 💰


In [1]:
import copy
import pandas as pd
import numpy as np
import random
import math
import scipy
from scipy import stats
from statistics import mean
from statistics import harmonic_mean
from collections import defaultdict


from rich import print
from rich.console import Console
from rich.table import Table

/tmp/ipykernel_1484235/2253750813.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def generate_TCM():
    # Set seed for reproducibility
    random.seed(11)

    # Initialize depots and their capacities
    # Number of Depots = 20 and Capacities = random integer between 100 and 150
    no_of_depots = 20
    depot_capacities = [random.randint(50, 150) for _ in range(no_of_depots)]
    df_demand = {'D{}'.format(i+1): cap for i, cap in enumerate(depot_capacities)}
    total_demand = sum(depot_capacities)
    
    # Calculate total supply

    # Generate supply data
    df_supply = {i: 1 for i in range(total_demand)}

    total_supply = sum(df_supply.values())
    
    np.random.seed(11)
    # Generate random dead kilometre matrix
    df_dkm = pd.DataFrame(np.random.uniform(0.5, 20, size=(total_supply, no_of_depots))).astype(float)
    
    random.seed(11)

    # Generate vehicle age
    vehicle_ages = [random.randint(1, 3) for _ in range(total_supply)]
    age = {i: age for i, age in enumerate(vehicle_ages)}

    # Generate kpl (kilometers per liter) and co2 emissions based on age
    random.seed(11) # Use the same random seed
    kpl, co2_emissions = [], []
    for v_age in vehicle_ages:
        if v_age == 1:
            kpl.append(random.uniform(5.1, 6))
            co2_emissions.append(round(random.uniform(515, 524), 2))
        elif v_age == 2:
            kpl.append(random.uniform(4.1, 5))
            co2_emissions.append(round(random.uniform(525, 534), 2))
        else:
            kpl.append(random.uniform(3.1, 4))
            co2_emissions.append(round(random.uniform(535, 540), 2))

    # Calculate fuel consumption, co2 cost and Depot Operating Cost
    fc_kpl = {i: (111 / kpl_val) for i, kpl_val in enumerate(kpl)}
    co2_c = {i: (0.00118642 * co2_val) for i, co2_val in enumerate(co2_emissions)}

    random.seed(11)
    df_doc = [random.uniform(50, 100) for _ in range(no_of_depots)]
    
    # Convert fc_kpl and co2_c dictionaries to pandas Series for efficient operations
    fc_kpl_series = pd.Series(fc_kpl)
    co2_c_series = pd.Series(co2_c)

    # Calculate the distance-based costs (fuel consumption and CO2 emissions) for each depot-destination pair
    distance_based_costs = df_dkm.multiply(fc_kpl_series, axis='index') + df_dkm.multiply(co2_c_series, axis='index')

    # Convert doc list to a pandas Series and align it with the columns of df_dkm for broadcasting
    doc_series = pd.Series(df_doc, index=df_dkm.columns)

    # Add the fixed depot operation costs to the distance-based costs
    # Broadcasting ensures that doc_series values are added to each row corresponding to the depot
    df_tdkom = distance_based_costs.add(doc_series, axis='columns')
    
    if input("Do you want to display the details of the total cost matrix? (Yes/No): ").lower() == "yes": display_TCM(no_of_depots, depot_capacities, total_demand, total_supply,df_supply, df_demand, df_dkm, kpl, co2_emissions, fc_kpl, co2_c, df_doc, df_tdkom)
    
    df_tdkom.columns = [("D"+str(i)) for i in range(1,21)]
    dk = df_tdkom.transpose()
    costs_t = dk.to_dict()
    cols = sorted(df_demand.keys())
    costs1=copy.deepcopy(costs_t)
    
    return costs_t, df_supply, df_demand, costs1, df_tdkom


In [3]:
def display_TCM(no_of_depots, depot_capacities, total_demand, total_supply,df_supply, df_demand, df_dkm, kpl, co2_emissions, fc_kpl, co2_c, df_doc, df_tdkom):
    # Initialize the Rich console
    console = Console()
    
    # Create a new table
    table = Table(show_header=True, header_style="#a2d2ff")
    table.add_column("Depot", style="dim", width=12)
    table.add_column("Capacity")

    # Populate the table with depot capacities
    for depot, capacity in df_demand.items():
        table.add_row(depot, str(capacity))

    # Print the table
    console.print(table)

    # Print total depot capacities
    console.print(f"Total Depot Capacities: [bold]{total_demand}[/bold]", style="#b0efeb")

    console = Console()

    table = Table(show_header=True, header_style="bold cyan")
    table.add_column("Bus", style="#a9def9")
    table.add_column("Value", style="#ffcad4")

    for key, value in list(df_supply.items())[:5]:
        table.add_row(str(key), str(value))

    table.add_row("...", "...")
    table.add_row(str(total_supply - 1), str(df_supply[total_supply - 1]))
    console.print(table)


    table = Table(show_header=True, header_style="bold #cdb4db")

    for i in range(5):
        table.add_column(f"Depot {i+1}", style="#a9def9")

    table.add_column("...", style="#a9def9")
    table.add_column(f"Depot {no_of_depots}", style="#a9def9")


    for index, row in df_dkm.head(5).iterrows():
        row_values = [f"{value:.2f}" for value in row[:5]]
        table.add_row(*row_values, "...", str(round(df_dkm[no_of_depots-1][index], 2)))
        
    table.add_row("...", "...","...", "...","...", "...", "...")
    for index, row in df_dkm.tail(2).iterrows():
        row_values = [f"{value:.2f}" for value in row[:5]]
        table.add_row(*row_values, "...", str(round(df_dkm[no_of_depots-1][index], 2)))
    console.print(table)

    table = Table(show_header=True, header_style="bold #ffb5a7")
    table.add_column("Vehicle ID", style="dim", width=12)
    table.add_column("KPL", style="#b0efeb")
    table.add_column("CO2 Emissions", style="#a9def9")
    table.add_column("Fuel Consumption (FC/KPL)", style="#cdb4db")
    table.add_column("CO2 Cost", style="#ffcad4")

    # Populate the table with vehicle data
    for i in range(5):
        table.add_row(
            str(i),
            "{:.2f}".format(kpl[i]),
            "{:.2f}".format(co2_emissions[i]),
            "{:.2f}".format(fc_kpl[i]),
            "{:.5f}".format(co2_c[i])
        )

    table.add_row("...", "...","...", "...","...")
    table.add_row("...", "...","...", "...","...")

    for i in range(total_supply-5, total_supply):
        table.add_row(
            str(i),
            "{:.2f}".format(kpl[i]),
            "{:.2f}".format(co2_emissions[i]),
            "{:.2f}".format(fc_kpl[i]),
            "{:.5f}".format(co2_c[i])
        )
        
    console.print(table)

    table = Table(show_header=True, header_style="bold cyan")
    table.add_column("Depot", style="#a9def9")
    table.add_column("Depot Operating Cost", style="#ffcad4")

    for key, value in enumerate(df_doc[:3]):
        table.add_row(str(key), str(value))

    table.add_row("...", "...")

    for index, value in enumerate(df_doc[-3:], start=no_of_depots - 3):
        table.add_row(str(index), str(value))
        
    console.print(table)

    table = Table(show_header=True, header_style="bold #cdb4db", title="[bold]🚌 Bus Depot Transportation Problem 🚌[/bold]")
    table.add_column("", style="#a9def9", justify="center")
    for i in range(5):
        table.add_column(f"Depot {i+1}", style="#a9def9", justify="center")
        
    table.add_column("...", style="#a9def9")
    table.add_column(f"Depot {no_of_depots}", style="#a9def9", justify="center")
    table.add_column("Supply ▼", style="#a9def9", justify="center")


    for index, row in df_tdkom.head(10).iterrows():
        row_values = [f"{value:.2f}" for value in row[:5]]
        table.add_row(f"[#ffcad4]Bus {index+1}[/#ffcad4]",*row_values, "...", str(round(df_tdkom[no_of_depots-1][index], 2)),"1")
        
    table.add_row("...", "...","...", "...","...", "...", "...", "...",'.')
    table.add_row("...", "...","...", "...","...", "...", "...", "...",'.')
    table.add_row("...", "...","...", "...","...", "...", "...", "...",'.')

    for index, row in df_tdkom.tail(10).iterrows():
        row_values = [f"{value:.2f}" for value in row[:5]]
        table.add_row(f"[#ffcad4]Bus {index+1}[/#ffcad4]",*row_values, "...", str(round(df_tdkom[no_of_depots-1][index], 2)),"1")
        
    table.add_row("","───────", "───────","───────", "───────","───────", "", "───────","────────")
    table.add_row("[#ffcad4]Demand ►[/#ffcad4]",*[str(value) for value in list(df_demand.values())[:5]],'...', str(list(df_demand.values())[-1]),str(total_supply))
    console.print(table)

In [4]:
def generate_TOCM():
    # Set seed for reproducibility
    random.seed(11)

    # Initialize depots and their capacities
    # Number of Depots = 20 and Capacities = random integer between 100 and 150
    no_of_depots = 20
    depot_capacities = [random.randint(50, 150) for _ in range(no_of_depots)]
    df_demand = {'D{}'.format(i+1): cap for i, cap in enumerate(depot_capacities)}
    total_demand = sum(depot_capacities)
    
    # Calculate total supply

    # Generate supply data
    df_supply = {i: 1 for i in range(total_demand)}

    total_supply = sum(df_supply.values())
    
    np.random.seed(1)
    # Generate random dead kilometre matrix
    df_dkm = pd.DataFrame(np.random.uniform(0.5, 20, size=(total_supply, no_of_depots))).astype(float)
    
    random.seed(11)

    # Generate vehicle age
    vehicle_ages = [random.randint(1, 3) for _ in range(total_supply)]
    age = {i: age for i, age in enumerate(vehicle_ages)}

    # Generate kpl (kilometers per liter) and co2 emissions based on age
    random.seed(11) # Use the same random seed
    kpl, co2_emissions = [], []
    for v_age in vehicle_ages:
        if v_age == 1:
            kpl.append(random.uniform(5.1, 6))
            co2_emissions.append(round(random.uniform(515, 524), 2))
        elif v_age == 2:
            kpl.append(random.uniform(4.1, 5))
            co2_emissions.append(round(random.uniform(525, 534), 2))
        else:
            kpl.append(random.uniform(3.1, 4))
            co2_emissions.append(round(random.uniform(535, 540), 2))

    # Calculate fuel consumption, co2 cost and Depot Operating Cost
    fc_kpl = {i: (111 / kpl_val) for i, kpl_val in enumerate(kpl)}
    co2_c = {i: (0.00118642 * co2_val) for i, co2_val in enumerate(co2_emissions)}

    random.seed(11)
    df_doc = [random.uniform(50, 100) for _ in range(no_of_depots)]
    
    # Convert fc_kpl and co2_c dictionaries to pandas Series for efficient operations
    fc_kpl_series = pd.Series(fc_kpl)
    co2_c_series = pd.Series(co2_c)

    # Calculate the distance-based costs (fuel consumption and CO2 emissions) for each depot-destination pair
    distance_based_costs = df_dkm.multiply(fc_kpl_series, axis='index') + df_dkm.multiply(co2_c_series, axis='index')

    # Convert doc list to a pandas Series and align it with the columns of df_dkm for broadcasting
    doc_series = pd.Series(df_doc, index=df_dkm.columns)

    # Add the fixed depot operation costs to the distance-based costs
    # Broadcasting ensures that doc_series values are added to each row corresponding to the depot
    df_tdkom = distance_based_costs.add(doc_series, axis='columns')
    
            
    #if input("Do you want to display the details of the total cost matrix? (Yes/No): ").lower() == "yes": display_TCM(no_of_depots, depot_capacities, total_demand, total_supply,df_supply, df_demand, df_dkm, kpl, co2_emissions, fc_kpl, co2_c, df_doc, df_tdkom)

    # Generating TOCM
    df_tdkom.columns = [("D"+str(i)) for i in range(1,21)]
    dk = df_tdkom.transpose()
    costs_t = dk.to_dict()
    cols = sorted(df_demand.keys())
    costs1=copy.deepcopy(costs_t)

    costs_tocm = copy.deepcopy(costs_t)
    costs2=copy.deepcopy(costs_t)
    costs3=copy.deepcopy(costs_t)
    for i in df_supply:
        mi=min(costs_t[i].values())
        # print(costs_t[i])
        # print(mi)
        for j in costs2[i]:
            costs2[i][j]-=mi
    # print(costs2)
    for i in df_demand :
        mi=10000
        for j in df_supply:
            if costs_t[j][i]<mi :
                mi=costs_t[j][i]
        for j in df_supply:
            costs3[j][i]=costs3[j][i]-mi 
    # print(costs3)

    for i in df_demand:
        for j in df_supply:
            costs_tocm[j][i]= costs2[j][i]+costs3[j][i]

    return costs_tocm, df_supply, df_demand, costs1, df_tdkom


In [ ]:
def SSM(costs,supply,demand):
    
    cost_f = copy.deepcopy(costs)
    df_sup =copy.deepcopy(supply)
    df_dem =copy.deepcopy(demand)

    res = dict((k, defaultdict(int)) for k in cost_f)
    g = {}
    for x in df_dem:
        g[x] = sorted(cost_f.keys(), key=lambda g: cost_f[g][x])

    for x in df_dem:
        res[g[x][0]][x] = df_dem[x]

    ## Deleting Rows if Supply is Met after First Allocation!

    #from operator import*
    for x in res:
        if list(res[x].values()) == []:
            #print('Null is:',x)
            continue
        else:
            #print(x)
            result = 0
            for i in list(res[x].values()):
                result += i
                #result = add(i, result)
            #print(result)
            if result == df_sup[x]:
                del df_sup[x]
                del cost_f[x]
        #print(x)

    while(df_sup):
        g = {}
        for x in df_dem:
            g[x] = sorted(cost_f.keys(), key=lambda g: cost_f[g][x])

        #### Step 3: To check the ER
        #from operator import*
        ER = []
        for x in df_sup:
            #### When only one cell is allocated a value in an ER
            if len(res[x].values()) == 1:
                if list(res[x].values())[0] > df_sup[x]:
                    ER.append(x)
                #print(ER)
            else:
                #### When multiple cells are allocated values in ER 
                result = 0
                for i in list(res[x].values()):
                    #result = add(i, result)
                    result += i
                if result > df_sup[x]:
                    ER.append(x)

                #print('Multiple cells:')
                #print(result,x)

        ## Step 4: To Calculate Differences for every cell in ER
        Diff = {}
        for x in ER:
            for y in res[x]:
                if len(g[y]) !=1:
                    if res[x][y] !=0:
                        m = 1
                        while g[y][m] in ER:
                            m += 1
                    #print(costs_t[g[y][0]][y], costs_t[g[y][m]][y])
                    Diff.setdefault(x, {}).setdefault(y, abs(cost_f[g[y][0]][y] - cost_f[g[y][m]][y]))

                elif len(g[y]) ==1:
                    Diff.setdefault(x, {}).setdefault(y, abs(cost_f[g[y][0]][y]))

        #print(Diff)

        SD = {key: min(val.values()) for key, val in Diff.items()}

        SDiff = min(SD.values())

        ### Counting occurrences of SDiff
        count = 0
        for key, value in SD.items():
            if value == SDiff:
                count += 1

        ### Step 5: For two/more SDiff with same value, choosing cell with largest allocation unit == (Will Always be SLC!)
        if count > 1:
            All = {}
            for k,v in Diff.items():
                #print(v)
                for (m,o) in v.items():
                    if o == SDiff:
                        All[k] = m

            max_val = 0
            for (key,value) in All.items():
                if res[key][value] > max_val:
                    max_val = res[key][value]
                    new_key = key
                    sub_key = value
                    #print(new_key,sub_key, res[new_key][sub_key])
                else:
                    continue

            new_value = SDiff
            #print(new_value, new_key, sub_key)

        if count == 1:
            new_value, new_key = min((value, key) for key, value in SD.items())

            key_list = list(Diff[new_key].keys())
            val_list = list(Diff[new_key].values())

            position = val_list.index(new_value)
            sub_key = key_list[position]

            #print(new_value, new_key, sub_key)

        ## If FLC Row cannot be satisfied & has more than one allocation:
        ## ~~ To ensure SLC must not be from an ER!
        if (len(Diff[new_key]) > 1):
            res_f = 0
            #for x in list(res[g[sub_key][0]].values()):
            for x in list(res[new_key].values()):
                res_f +=x

            ### Adding allocations in SLC Row
            x = 1
            while g[sub_key][x] in ER:
                x += 1

            ## Replacing 1's with 'x'
            #if res_f > df_supply[g[sub_key][0]]:
            if res_f > df_sup[new_key]:
                result = 0
                for x in list(res[g[sub_key][x]].values()):
                    result +=x

                ### If possible, satisfying the SLC Row
                if result < df_sup[g[sub_key][x]]:
                    m = df_sup[g[sub_key][x]] - result
                    res[g[sub_key][x]][sub_key] = m
                    res[new_key][sub_key] -= m
                    if res[new_key][sub_key] == 0:
                        del res[new_key][sub_key]
                    #res[g[sub_key][0]][sub_key] -= m
                    #if res[g[sub_key][0]][sub_key] == 0:
                    #    del res[g[sub_key][0]][sub_key]

                ### If it is not possible to satisfy the SLC Row, move all All.units from current FLC to SLC
                if result > df_sup[g[sub_key][x]]:
                    #m = res[g[sub_key][0]][sub_key]
                    m = res[new_key][sub_key]
                    res[g[sub_key][x]][sub_key] += m
                    del res[new_key][sub_key]
                    #del res[g[sub_key][0]][sub_key]

        ## If FLC Row can be satisfied & has only one allocation:
        if (len(Diff[new_key])==1): 

            ###~~~~~~~~~~~ CORRECTION !!!!!

            x = 1
            while g[sub_key][x] in ER:
                x += 1
            #print(x)
            if res[new_key][sub_key] == 1:
                res[g[sub_key][x]][sub_key] = res[new_key][sub_key]
                del res[new_key][sub_key]
            else:
                res[g[sub_key][x]][sub_key] = res[new_key][sub_key] - df_sup[new_key]
                res[new_key][sub_key] = df_sup[new_key]

        ### Adding up Cells in Rows
        result_1 = 0
        for i in list(res[new_key].values()):
            result_1 += i
        #print(result_1)

        result_2 = 0
        for i in list(res[g[sub_key][x]].values()):
            result_2 += i
        #print(result_2)

        if result_1 == df_sup[new_key]:
            del df_sup[new_key]
            del cost_f[new_key]

        if result_2 == df_sup[g[sub_key][x]]:
            del df_sup[g[sub_key][x]]
            del cost_f[g[sub_key][x]]

    cost = 0
    #costs1=copy.deepcopy(costs_t)
    cols = sorted(df_demand.keys())
    
    for g in sorted(costs1):
        # print (g, " ",)
        for n in cols:
            y = res[g][n]
            if y != 0:
                pass
                # print (y,)
            cost += y * costs1[g][n]
            # print ("  ",)
        # print(" ")
    print ("Total Cost yielded by SSM = ", cost)

In [ ]:
def BCE(costs,supply,demand):
    
    cost_f = copy.deepcopy(costs)
    df_sup =copy.deepcopy(supply)
    df_dem =copy.deepcopy(demand)
    
    res = dict((k, defaultdict(int)) for k in cost_f)
    g = {}
    for x in df_dem:
        g[x] = sorted(cost_f.keys(), key=lambda g: cost_f[g][x])

    for x in df_dem:
        res[g[x][0]][x] = df_dem[x]
        
    ## Deleting Rows if Supply is Met after First Allocation!

    # from operator import*
    for x in res:
        if list(res[x].values()) == []:
            #print('Null is:',x)
            continue
        else:
            #print(x)
            result = 0
            for i in list(res[x].values()):
                result += i
                #result = add(i, result)
            #print(result)
            if result == df_sup[x]:
                del df_sup[x]
                del cost_f[x]
        #print(x)
        
    while(df_sup):
        g = {}
        for x in df_dem:
            g[x] = sorted(cost_f.keys(), key=lambda g: cost_f[g][x])

        #### Step 3: To check the ER
        #from operator import*
        ER = []
        for x in df_sup:
            #### When only one cell is allocated a value in an ER
            if len(res[x].values()) == 1:
                if list(res[x].values())[0] > df_sup[x]:
                    ER.append(x)
                #print(ER)
            else:
                #### When multiple cells are allocated values in ER 
                result = 0
                for i in list(res[x].values()):
                    #result = add(i, result)
                    result += i
                if result > df_sup[x]:
                    ER.append(x)


        ## Step 4: To Calculate Differences for every cell in ER
        Diff = {}
        for x in ER:
            for y in res[x]:
                if len(g[y]) !=1:
                    if res[x][y] !=0:
                        m = 1
                        while g[y][m] in ER:
                            m += 1
                    #print(costs_t[g[y][m]][y], costs_t[g[y][0]][y])
                    Diff.setdefault(x, {}).setdefault(y, cost_f[g[y][m]][y] - cost_f[g[y][0]][y])         

                elif len(g[y]) ==1:
                    Diff.setdefault(x, {}).setdefault(y, cost_f[g[y][0]][y])

        #print(Diff)

        SD = {key: min(val.values()) for key, val in Diff.items()}

        SDiff = min(SD.values())

        ### Counting occurrences of SDiff
        count = 0
        for key, value in SD.items():
            if value == SDiff:
                count += 1

        ### Step 5: For two/more SDiff with same value, choosing cell with largest allocation unit!
        if count > 1:
            All = {}
            for k,v in Diff.items():
                #print(v)
                for (p,o) in v.items():
                    if o == SDiff:
                        All[k] = p

            max_val = 0
            for (key,value) in All.items():
                if res[key][value] > max_val:
                    max_val = res[key][value]
                    new_key = key
                    sub_key = value
                    #print(new_key,sub_key, res[new_key][sub_key])
                else:
                    continue

            new_value = SDiff
            #print(new_value, new_key, sub_key)

        if count == 1:
            new_value, new_key = min((value, key) for key, value in SD.items())

            key_list = list(Diff[new_key].keys())
            val_list = list(Diff[new_key].values())

            position = val_list.index(new_value)
            sub_key = key_list[position]

            #print(new_value, new_key, sub_key)

        FLR = new_key
        #FLR

        m = 1
        while g[sub_key][m] in ER:
            m += 1
        #print(m)
        SLR = g[sub_key][m]
        #SLR

        TCFLR = 0
        for i in list(cost_f[FLR].values()):
            TCFLR += i

        TCSLR = 0
        for i in list(cost_f[SLR].values()):
            TCSLR += i

        ### Step 9:
        if TCFLR > TCSLR:
            ## Satisfy FLR & keep the rest in SLC
            if res[FLR][sub_key] == 1:
                res[SLR][sub_key] = res[FLR][sub_key]
                del res[FLR][sub_key]
            else:
                res[SLR][sub_key] = res[FLR][sub_key] - df_sup[FLR]
                res[FLR][sub_key] = df_sup[FLR]


        ### Step 9:
        if TCFLR < TCSLR:
            ## Satisfy SLR & Keep the rest in FLC
            if res[FLR][sub_key] == 1:
                res[SLR][sub_key] = res[FLR][sub_key]
                del res[FLR][sub_key]
            else:
                res[FLR][sub_key] = res[FLR][sub_key] - df_sup[FLR]
                res[SLR][sub_key] = df_sup[SLR]


        result_1 = 0
        for i in list(res[FLR].values()):
            result_1 += i
        #print(result_1)

        result_2 = 0
        for i in list(res[SLR].values()):
            result_2 += i
        #print(result_2)

        if result_1 == df_sup[FLR]:
            del df_sup[FLR]
            del cost_f[FLR]

        if result_2 == df_sup[SLR]:
            del df_sup[SLR]
            del cost_f[SLR]
            
    cost = 0
    cols = sorted(df_demand.keys())
    for g in sorted(costs1):
        # print (g, " ",)
        for n in cols:
            y = res[g][n]
            if y != 0:
                pass
                # print (y,)
            cost += y * costs1[g][n]
            # print ("  ",)
        # print(" ")
    print ("Total Cost yielded by BCE = ", cost)

In [ ]:
def JHM(costs,supply,demand):
    
    cost_f = copy.deepcopy(costs)
    df_sup =copy.deepcopy(supply)
    df_dem =copy.deepcopy(demand)
    
    res = dict((k, defaultdict(int)) for k in cost_f)
    g = {}
    for x in df_sup:
        g[x] = sorted(cost_f[x].keys(), key=lambda g: cost_f[x][g])
    for x in df_dem:
        g[x] = sorted(cost_f.keys(), key=lambda g: cost_f[g][x])
    # print(costs1)
    flag=0
    while g:
        l=[]
        for x in df_dem:

            # print(demand[x],supply[g[x][0]])
            if df_dem[x]> df_sup[g[x][0]]:
                l.append(x)
                if len(l)==2:
                    break
        # break
        mi=100000
        dem="D"
        # print(l)
        if len(l)==0:
            for x in df_dem:
                t=g[x][0]
                res[t][x]+=df_dem[x]
                df_sup[t]-=df_dem[x]
                df_dem[x]=0
                if df_sup[t]==0:
                    if df_sup[t] == 0:
                        for k, n in df_dem.items():
                            if n != 0:
                                g[k].remove(t)
                    del g[t]
                    del df_sup[t]
                for k, n in df_sup.items():
                    if n != 0:
                        g[k].remove(x)
                del g[x]
                del df_dem[x]
                break
            continue
        for x in l:

            diff = cost_f[g[x][1]][x]-cost_f[g[x][0]][x] if len(g[x])>1 else cost_f[g[x][0]]
            if diff < mi:
                mi=diff
                dem=x
        x=dem 

        # print(x,mi)
        # break

        while(df_dem[x]>0):

            v=min(df_dem[x],df_sup[g[x][0]])
            df_dem[x]-=v
            df_sup[g[x][0]]-=v
            res[g[x][0]][x]+=v
            t=g[x][0]
            # print(t)

            if df_sup[t]==0:
                if df_sup[t] == 0:
                    for k, n in df_dem.items():
                        if n != 0:
                            g[k].remove(t)
                del g[t]
                del df_sup[t]
        for k, n in df_sup.items():
            if n != 0:
                    g[k].remove(x)
        del g[x]
        del df_dem[x]

    cost = 0
    cols = sorted(df_demand.keys())
    for g in sorted(costs1):
        # print (g, " ",)
        for n in cols:
            y = res[g][n]
            if y != 0:
                pass
                # print (y,)
            cost += y * costs1[g][n]
            # print ("  ",)
        # print(" ")
    print ("Total Cost yielded by JHM = ", cost)


In [ ]:
def RAM(costs,supply,demand):
    
    cost_f = copy.deepcopy(costs)
    df_sup =copy.deepcopy(supply)
    df_dem =copy.deepcopy(demand)
    
    g = {}
    for x in df_sup:
        g[x] = sorted(cost_f[x].keys(), key=lambda g: cost_f[x][g], reverse=True)
    for y in df_dem:
        g[y] = sorted(cost_f.keys(), key=lambda g: cost_f[g][y] , reverse=True)
    
    computed = {}

    for x in df_sup:
        computed[x]={}
        for y in df_dem:
            computed[x][y]= - (cost_f[x][y] - cost_f[x][g[x][0]] - cost_f[g[y][0]][y])
    
    max_computed = {}
    for x in df_sup:
        max_computed[x] = sorted(computed[x].keys(), key=lambda g: computed[x][g], reverse=True)

    res = dict((k, defaultdict(int)) for k in cost_f)
    
    while g:
    
        old_d = "D"
        old_s = "S"

        ### Beginning of a loop
        # cleaning
        old_max = 0
        for x in df_sup:
            if computed[x][max_computed[x][0]] > old_max:
                #print(computed[x][max_computed[x][0]])
                old_max = computed[x][max_computed[x][0]]
                old_d = max_computed[x][0]
                old_s = x

        #print(old_max)
        #print(old_d)
        #print(old_s)

        # added to results matrix

        res[old_s][old_d] += 1

        df_dem[old_d] -= 1
        df_sup[old_s] -= 1


        ### Simultaneously ensures whether supply/demand gets cut
        if df_sup[old_s] == 0:
            for y in df_dem:
                if g[y][0] == old_s:
                    for x in df_sup:
                        if len(g[y])>1:
                            computed[x][y] = computed[x][y] - cost_f[g[y][0]][y] + cost_f[g[y][1]][y]


        if df_dem[old_d] == 0:
            for x in df_sup:
                #go through all demand points within this supply and update computed cost
                if g[x][0] == old_d:
                    for y in df_dem:
                        if len(g[x])>1:
                            computed[x][y] = computed[x][y] - cost_f[x][g[x][0]] + cost_f[x][g[x][1]]

        if df_sup[old_s] == 0:
            del g[old_s]
            del df_sup[old_s]
            del computed[old_s]
            #del max_computed[old_s]
            for k, n in df_dem.items():
                #if n == 0:
                g[k].remove(old_s)
                #max_computed[k].remove(old_s)

        if df_dem[old_d] == 0:
            del g[old_d]
            del df_dem[old_d]
            for k, n in df_sup.items():
                g[k].remove(old_d)
              # print("removing" + str(k) + "and" + str(old_d))
                #max_computed[k].remove(old_d)
                del computed[k][old_d]
              # print(max_computed[k])

        max_computed = {}
        for x in df_sup:
            max_computed[x] = sorted(computed[x].keys(), key=lambda g: computed[x][g], reverse=True)
            
    cost = 0
    for x in cost_f.keys():
        for y in cost_f[x].keys():
            cost += costs1[x][y] * res[x][y]

    print("Total Cost yielded by JHM = ", cost)

In [ ]:
def KSM(costs,supply,demand):
    
    costs4=copy.deepcopy(costs)
    costs2=copy.deepcopy(costs)
    costs3=copy.deepcopy(costs)
    ma=0
    
    demand_d = demand.copy()
    supply_d = supply.copy()
    demand_s = demand.copy()
    supply_s = supply.copy()
    
    cold = sorted(demand.keys())
    cols = sorted(demand.keys())
    
    ### Weighted Transportation Matrix by Demand (wcd)

    res = dict((k, defaultdict(int)) for k in costs4)
    rij={}
    for x in supply_d:
        dd={}
        for y in demand_d:
            dd[y]=0
        rij[x]=dd

    # Proportional Demand Matrix
    for x in supply_d:
        for y in demand_d:
            rij[x][y]= demand_d[y]/supply_d[x]

    # print(rij)
    for x in supply_d:
        for y in demand_d:
            rij[x][y]*=costs4[x][y]
            
    while supply_d or demand_d:
        mi=10000000000000
        l=[]
        for x in supply_d:
            for y in demand_d:
                if rij[x][y]<mi:
                    mi=rij[x][y]
                    l=[x,y]

        sup=l[0]
        dem=l[1]
        v=min(supply_d[sup],demand_d[dem])
        demand_d[dem]-=v
        supply_d[sup]-=v
        res[sup][dem]+=v
        if supply_d[sup]==0:
            del supply_d[sup]
        if demand_d[dem]==0:
            del demand_d[dem]

    cost_d = 0
    for g in sorted(costs4):
      # print (g, "\t",)
      for n in cold:
        y = res[g][n]
        if y != 0:
            pass
          # print (y,)
        cost_d += y * costs1[g][n]
        # print ("\t",)
      # print()
    # print ("\n\nTotal Cost = ", cost)
    
    ### Weighted Transportation Matrix by Supply (wcs)

    res_s = dict((k, defaultdict(int)) for k in costs2)
    rij_s={}
    for x in supply_s:
        dd={}
        for y in demand_s:
            dd[y]=0
        rij_s[x]=dd

    for x in supply_s:
        for y in demand_s:
            rij_s[x][y]= supply_s[x]/demand_s[y]
    # print(rij)
    for x in supply_s:
        for y in demand_s:
            rij_s[x][y]*=costs2[x][y]
            
    while supply_s or demand_s:
        mi=10000000000000
        ls=[]
        for x in supply_s:
            for y in demand_s:
                if rij_s[x][y]<mi:
                    mi=rij_s[x][y]
                    ls=[x,y]

        sup_s=ls[0]
        dem_s=ls[1]
        v1=min(supply_s[sup_s],demand_s[dem_s])
        demand_s[dem_s]-=v1
        supply_s[sup_s]-=v1
        res_s[sup_s][dem_s]+=v1
        if supply_s[sup_s]==0:
            del supply_s[sup_s]
        if demand_s[dem_s]==0:
            del demand_s[dem_s]

    #print(demand_s,supply_s)
    
    cost_s = 0
    for g in sorted(costs2):
      # print (g, "\t",)
      for n in cols:
        y = res_s[g][n]
        if y != 0:
            pass
          # print (y,)
        cost_s += y * costs1[g][n]
        # print ("\t",)
      # print()
    
    print ("Total Cost yielded by KSM = ", min(cost_d,cost_s))

In [ ]:
def MT(costs,supply,demand):
    
    cost_f = copy.deepcopy(costs)
    df_sup =copy.deepcopy(supply)
    df_dem =copy.deepcopy(demand)
    
    res = dict((k, defaultdict(int)) for k in cost_f)
    g = {}
    for x in df_sup:
        g[x] = sorted(cost_f[x].keys(), key=lambda g: cost_f[x][g])
    for x in df_dem:
        g[x] = sorted(cost_f.keys(), key=lambda g: cost_f[g][x])
    # print(costs1)
    flag=0
    
    while df_sup and df_dem:
        d = {}
        # print(df_demand,df_supply)

        s = {}
        for x in df_sup:
            s[x] = 0
            for y in df_dem:
                s[x]=s[x] + cost_f[x][y]- cost_f[x][g[x][0]]

        mp = max(list(s.values()))
        mi=1000000
        l=[]
        for x in df_sup :
            if s[x]==mp:
                l.append(x)
                if cost_f[x][g[x][0]]<mi:
                    mi=cost_f[x][g[x][0]]
        sup="S"
        ma=0
        # print(demand)
        for x in l :
            # print(g[x][0])
            if cost_f[x][g[x][0]]==mi and df_dem[g[x][0]]>=ma:
                sup=x 
        # print(costs[sup][g[sup][0]], len(df_supply))
        if cost_f[sup][g[sup][0]]!=0 or len(df_sup)==1:
            # print("here")
            v=min(df_sup[sup],df_dem[g[sup][0]])
            # print(sup,demand,v)
            df_dem[g[sup][0]]-=v
            df_sup[sup]-=v 
            res[sup][g[sup][0]]+=v 
            dem = g[sup][0]
            # print(sup,dem,v)
            if df_dem[dem]==0:
                for k, n in df_sup.items():
                    if n != 0:
                        g[k].remove(dem)
                del g[dem]
                del df_dem[dem]
            if df_sup[sup]==0:
                for k, n in df_dem.items():
                    if n != 0:
                        g[k].remove(sup)
                del g[sup]
                del df_sup[sup]

        else :
            mind="S"
            ma=-1
            # print(sup)
            for x in df_sup :
                if s[x]>ma and x!=sup :
                    ma=s[x]
                    mind=x
            gv1=0
            # print(mind,"this")
            for y in df_dem:
                if cost_f[sup][y] > cost_f[mind][y]:
                    gv1+=1 
                else:
                    gv1-=1
            if gv1>=0 :
                v=min(df_sup[sup],df_dem[g[sup][0]])
                # print(sup,demand,v)
                df_dem[g[sup][0]]-=v
                df_sup[sup]-=v 
                res[sup][g[sup][0]]+=v 
                dem=g[sup][0]
                # print(sup,dem,v)
                if df_dem[dem]==0:
                    for k, n in df_sup.items():
                        if n != 0:
                            g[k].remove(dem)
                    del g[dem]
                    del df_dem[dem]
                if df_sup[sup]==0:
                    for k, n in df_dem.items():
                        if n != 0:
                            g[k].remove(sup)
                    del g[sup]
                    del df_sup[sup]
            else :
                sup=mind 
                v=min(df_sup[sup],df_dem[g[sup][0]])

                df_dem[g[sup][0]]-=v
                df_sup[sup]-=v 
                res[sup][g[sup][0]]+=v 
                dem=g[sup][0]
                # print(sup,dem,v)
                if df_dem[dem]==0:
                    for k, n in df_sup.items():
                        if n != 0:
                            g[k].remove(dem)
                    del g[dem]
                    del df_dem[dem]
                if df_sup[sup]==0:
                    for k, n in df_dem.items():
                        if n != 0:
                            g[k].remove(sup)
                    del g[sup]
                    del df_sup[sup]
        # break

    cost = 0
    cols = sorted(df_demand.keys())
    for g in sorted(costs1):
        # print (g, " ",)
        for n in cols:
            y = res[g][n]
            if y != 0:
                pass
                # print (y,)
            cost += y * costs1[g][n]
            # print ("  ",)
        # print(" ")
    
    print ("Total Cost yielded by MT = ", cost)

In [ ]:
def EDM(costs,supply,demand):
    
    cost_f = copy.deepcopy(costs)
    df_sup =copy.deepcopy(supply)
    df_dem =copy.deepcopy(demand)
    
    res = dict((k, defaultdict(int)) for k in cost_f)
    g = {}
    for x in df_dem:
        g[x] = sorted(cost_f.keys(), key=lambda g: cost_f[g][x])

    for x in df_sup:
        g[x] = sorted(cost_f[x].keys(), key=lambda g: cost_f[x][g])
        
    while g:
        EDR = {}
        for x in df_sup:
            EDR[x] = (cost_f[x][g[x][-1]] - cost_f[x][g[x][0]])  if len(g[x]) > 1 else cost_f[x][g[x][0]]

        EDC = {}
        for x in df_dem:
            EDC[x] = (cost_f[g[x][-1]][x] - cost_f[g[x][0]][x]) if len(g[x]) > 1 else (cost_f[g[x][0]][x])

        f = max(EDC, key=lambda n: EDC[n])
        t = max(EDR, key=lambda n: EDR[n])

        t, f = (f, g[f][0]) if EDC[f] >= EDR[t] else (g[t][0], t)

        v = min(df_sup[f], df_dem[t])

        res[f][t] += v
        df_dem[t] -= v

        if df_dem[t] == 0:
            for k, n in df_sup.items():
                if n != 0:
                    g[k].remove(t)
            del g[t]
            del df_dem[t]
        df_sup[f] -= v
        if df_sup[f] == 0:
            for k, n in df_dem.items():
                if n != 0:
                    g[k].remove(f)
            del g[f]
            del df_sup[f]
            
    cost = 0
    cols = sorted(df_demand.keys())
    for g in sorted(costs1):
        # print (g, "\t",)
        for n in cols:
            y = res[g][n]
            if y != 0:
                pass
                # print (y,)
            cost += y * costs1[g][n]
            # print ("\t",)
        # print()
    print ("Total Cost yielded by EDM = ", cost)

In [ ]:
def GMM(costs,supply,demand):
    
    cost_f = copy.deepcopy(costs)
    df_sup =copy.deepcopy(supply)
    df_dem =copy.deepcopy(demand)
    
    for i in df_sup:
        for j in df_dem:
            cost_f[i][j] = cost_f[i][j]*min(df_sup[i],df_dem[j])
    
    res = dict((k, defaultdict(int)) for k in cost_f)
    
    while (cost_f):
        sub_dict = {}
        min_value_n = 10000
        for i in df_sup:
            sub_dict = cost_f[i]
            #print(sub_dict)
            min_value, min_key = min((value, key) for key, value in sub_dict.items())
            #print(min_value, min_key)
            if min_value <= min_value_n:
                #print(min_value, min_value_n)
                min_value_n = min_value
                min_key_n = min_key
                sub_key = i
                continue

        res[sub_key][min_key_n] = min(df_sup[sub_key],df_dem[min_key_n])
        df_dem[min_key_n]-= res[sub_key][min_key_n]
        df_sup[sub_key]-= res[sub_key][min_key_n]

        if df_dem[min_key_n] == 0:
            del df_dem[min_key_n]
            for i in df_sup:
                del cost_f[i][min_key_n]

        if df_sup[sub_key] == 0:
            del df_sup[sub_key]
            del cost_f[sub_key]
            
    cost = 0
    cols = sorted(df_demand.keys())
    for g in sorted(costs1):
        # print (g, "\t",)
        for n in cols:
            y = res[g][n]
            if y != 0:
                pass
                # print (y,)
            cost += y * costs1[g][n]
            # print ("\t",)
        # print()
    print ("Total Cost = ", cost)

In [ ]:
def VAM(costs,supply,demand):
    
    cost_f = copy.deepcopy(costs)
    df_sup =copy.deepcopy(supply)
    df_dem =copy.deepcopy(demand)
    
    res = dict((k, defaultdict(int)) for k in cost_f)
    g = {}
    for x in df_sup:
        g[x] = sorted(cost_f[x].keys(), key=lambda g: cost_f[x][g])
    for x in df_demand:
        g[x] = sorted(cost_f.keys(), key=lambda g: cost_f[g][x])


    while g:
        d = {}
        for x in df_dem:
            d[x] = (cost_f[g[x][1]][x] - cost_f[g[x][0]][x]) if len(g[x]) > 1 else (cost_f[g[x][0]][x])
        s = {}
        for x in df_sup:
            s[x] = (cost_f[x][g[x][1]] - cost_f[x][g[x][0]])  if len(g[x]) > 1 else cost_f[x][g[x][0]]
        f = max(d, key=lambda n: d[n])
        t = max(s, key=lambda n: s[n])
        t, f = (f, g[f][0]) if d[f] >= s[t] else (g[t][0], t)
        v = min(df_sup[f], df_dem[t])
        res[f][t] += v
        df_dem[t] -= v

        if df_dem[t] == 0:
            for k, n in df_sup.items():
                if n != 0:
                    g[k].remove(t)
            del g[t]
            del df_dem[t]
        df_sup[f] -= v
        if df_sup[f] == 0:
            for k, n in df_dem.items():
                if n != 0:
                    g[k].remove(f)
            del g[f]
            del df_sup[f]

    cost = 0
    cols = sorted(df_demand.keys())
    for g in sorted(costs1):
        for n in cols:
            y = res[g][n]
            if y != 0:
                pass
            cost += y * costs1[g][n]
    print("Total Cost yielded by VAM = ", cost)

In [ ]:
def RCMCAM(costs,supply,demand):
    
    cost_f = copy.deepcopy(costs)
    df_sup =copy.deepcopy(supply)
    df_dem =copy.deepcopy(demand)
    
    res = dict((k, defaultdict(int)) for k in cost_f)
    g = {}
    for x in df_sup:
        g[x] = sorted(cost_f[x].keys(), key=lambda g: cost_f[x][g])
    for x in df_dem:
        g[x] = sorted(cost_f.keys(), key=lambda g: cost_f[g][x])

    while g:
        d = {}
        # print(df_supply,dic)
        for x in df_dem:
            d[x] = cost_f[g[x][0]][x]*min(df_dem[x],df_sup[g[x][0]]) if len(g[x]) > 1 else (cost_f[g[x][0]][x])*min(df_dem[x],df_sup[g[x][0]])
        s = {}
        for x in df_sup:
            s[x] = cost_f[x][g[x][0]]*min(df_sup[x],df_dem[g[x][0]])  if len(g[x]) > 1 else cost_f[x][g[x][0]]*min(df_sup[x],df_dem[g[x][0]])

        f = max(d, key=lambda n: d[n])
        t = max(s, key=lambda n: s[n])


        t, f = (f, g[f][0]) if d[f] >= s[t] else (g[t][0], t)


        v = min(df_sup[f], df_dem[t])
        res[f][t] += v
        df_dem[t] -= v

        if df_dem[t] == 0:
            for k, n in df_sup.items():
                if n != 0:
                    g[k].remove(t)
            del g[t]
            del df_dem[t]
        df_sup[f] -= v
        if df_sup[f] == 0:
            for k, n in df_dem.items():
                if n != 0:
                    g[k].remove(f)
            del g[f]
            del df_sup[f]

    cost = 0
    cols = sorted(df_demand.keys())
    for g in sorted(costs1):
        for n in cols:
            y = res[g][n]
            if y != 0:
                pass
            cost += y * costs1[g][n]
    print("Total Cost yielded by RCMCAM = ", cost)


In [ ]:
def RCWMCAM(costs,supply,demand):
    
    cost_f = copy.deepcopy(costs)
    df_sup =copy.deepcopy(supply)
    df_dem =copy.deepcopy(demand)
    
    res = dict((k, defaultdict(int)) for k in costs_t)
    g = {}
    for x in df_sup:
        g[x] = sorted(costs_t[x].keys(), key=lambda g: costs_t[x][g])
    for x in df_dem:
        g[x] = sorted(costs_t.keys(), key=lambda g: costs_t[g][x])

    while g:
        d = {}
        for x in df_dem:
            d[x] = (cost_f[g[x][1]][x] - cost_f[g[x][0]][x])*cost_f[g[x][0]][x]*min(df_dem[x],df_sup[g[x][0]]) if len(g[x]) > 1 else (cost_f[g[x][0]][x]*cost_f[g[x][0]][x])*min(df_dem[x],df_sup[g[x][0]])
        s = {}
        for x in df_sup:
            s[x] = (cost_f[x][g[x][1]] - cost_f[x][g[x][0]])*cost_f[x][g[x][0]]*min(df_sup[x],df_dem[g[x][0]])  if len(g[x]) > 1 else cost_f[x][g[x][0]]*cost_f[x][g[x][0]]*min(df_sup[x],df_dem[g[x][0]])

        f = max(d, key=lambda n: d[n])
        t = max(s, key=lambda n: s[n])


        t, f = (f, g[f][0]) if d[f] >= s[t] else (g[t][0], t)


        v = min(df_sup[f], df_dem[t])
        res[f][t] += v
        df_dem[t] -= v

        if df_dem[t] == 0:
            for k, n in df_sup.items():
                if n != 0:
                    g[k].remove(t)
            del g[t]
            del df_dem[t]
        df_sup[f] -= v
        if df_sup[f] == 0:
            for k, n in df_dem.items():
                if n != 0:
                    g[k].remove(f)
            del g[f]
            del df_sup[f]

    cost = 0
    cols = sorted(df_demand.keys())
    for g in sorted(costs1):
        for n in cols:
            y = res[g][n]
            if y != 0:
                pass
            cost += y * costs1[g][n]
    
    print("Total Cost yielded by RCWMCAM = ", cost)

In [5]:
def WPCM1(costs,supply,demand):
    
    cost_f = copy.deepcopy(costs)
    df_sup =copy.deepcopy(supply)
    df_dem =copy.deepcopy(demand)
    mi = 10000
    
    res = dict((k, defaultdict(int)) for k in cost_f)
    g = {}
    for x in df_sup:
        g[x] = sorted(cost_f[x].keys(), key=lambda g: cost_f[x][g])
    for x in df_dem:
        g[x] = sorted(cost_f.keys(), key=lambda g: cost_f[g][x])
    flag=0
    while g:

        d = {}
        for x in df_dem:
            d[x] = (cost_f[g[x][1]][x] - cost_f[g[x][0]][x])* min(df_dem[x],df_sup[g[x][0]]) if len(g[x]) > 1 else (cost_f[g[x][0]][x])*min(df_dem[x],df_sup[g[x][0]])
        s = {}
        for x in df_sup:
            d[x] = (cost_f[x][g[x][1]] - cost_f[x][g[x][0]])*min(df_sup[x],df_dem[g[x][0]]) if len(g[x]) > 1 else cost_f[x][g[x][0]]*min(df_sup[x],df_dem[g[x][0]])

        for x in df_sup :
            for y in df_dem:
                if cost_f[x][y]*min(df_sup[x],df_dem[y])<mi :
                    mi= cost_f[x][y]*min(df_sup[x],df_dem[y])
        sd = max(d, key=lambda n: d[n])
        t,f="D","S"
        if df_dem.get(sd)!=None:
            t,f = sd, g[sd][0]
        else :
            t,f = g[sd][0],sd


        v = min(df_sup[f], df_dem[t])
        res[f][t] += v
        df_dem[t] -= v

        if df_dem[t] == 0:
            for k, n in df_sup.items():
                if n != 0:
                    g[k].remove(t)
            del g[t]
            del df_dem[t]
        df_sup[f] -= v
        if df_sup[f] == 0:
            for k, n in df_dem.items():
                if n != 0:
                    g[k].remove(f)
            del g[f]
            del df_sup[f]

    cost = 0
    cols = sorted(df_demand.keys())
    for g in sorted(costs1):
        for n in cols:
            y = res[g][n]
            if y != 0:
                pass
            cost += y * costs1[g][n]
    print ("Total Cost yielded by WPCM1 = ", cost)

In [ ]:
def WUPCM1(costs,supply,demand):
    
    cost_f = copy.deepcopy(costs)
    df_sup =copy.deepcopy(supply)
    df_dem =copy.deepcopy(demand)
    
    res = dict((k, defaultdict(int)) for k in cost_f)

    g = {}
    for x in df_sup:
        g[x] = sorted(cost_f[x].keys(), key=lambda g: cost_f[x][g])
    for x in df_dem:
        g[x] = sorted(cost_f.keys(), key=lambda g: cost_f[g][x])
        
    while g:
        d = {}
        # print(df_supply,dic)
        for x in df_dem:
            d[x] = ((cost_f[g[x][1]][x] - cost_f[g[x][0]][x])/df_dem[x])*min(df_dem[x],df_sup[g[x][0]]) if len(g[x]) > 1 else (cost_f[g[x][0]][x]/df_dem[x])*min(df_dem[x],df_sup[g[x][0]])
        s = {}
        for x in df_sup:
            s[x] = ((cost_f[x][g[x][1]] - cost_f[x][g[x][0]])/df_sup[x])*min(df_sup[x],df_dem[g[x][0]])  if len(g[x]) > 1 else (cost_f[x][g[x][0]]/df_sup[x])*min(df_sup[x],df_dem[g[x][0]])

        f = max(d, key=lambda n: d[n])
        t = max(s, key=lambda n: s[n])


        t, f = (f, g[f][0]) if d[f] >= s[t] else (g[t][0], t)


        v = min(df_sup[f], df_dem[t])
        res[f][t] += v
        df_dem[t] -= v

        if df_dem[t] == 0:
            for k, n in df_sup.items():
                if n != 0:
                    g[k].remove(t)
            del g[t]
            del df_dem[t]
        df_sup[f] -= v
        if df_sup[f] == 0:
            for k, n in df_dem.items():
                if n != 0:
                    g[k].remove(f)
            del g[f]
            del df_sup[f]

    cost = 0
    cols = sorted(df_demand.keys())
    # print(costs)
    for g in sorted(costs1):
        for n in cols:
            y = res[g][n]
            # print("YESS",y)
            if y != 0:
                pass
                # print (y,)
            cost += y * costs1[g][n]
            # print ("  ",)
    print("Total Cost yielded by WUPCM1 = ", cost)

In [ ]:
def WUPCM2(costs,supply,demand):
    
    cost_f = copy.deepcopy(costs)
    df_sup =copy.deepcopy(supply)
    df_dem =copy.deepcopy(demand)
    
    res = dict((k, defaultdict(int)) for k in cost_f)

    g = {}
    for x in df_sup:
        # print(x)
        # print(costs_t[x])
        g[x] = sorted(cost_f[x].keys(), key=lambda g: cost_f[x][g])
    for x in df_demand:
        g[x] = sorted(cost_f.keys(), key=lambda g: cost_f[g][x])

    while g:
        d = {}
        # print(df_supply,dic)
        for x in df_dem:
            d[x] = ((cost_f[g[x][1]][x] - cost_f[g[x][0]][x])/df_dem[x])*cost_f[g[x][0]][x] if len(g[x]) > 1 else (cost_f[g[x][0]][x]/df_dem[x])*cost_f[g[x][0]][x]
        s = {}
        for x in df_sup:
            s[x] = ((cost_f[x][g[x][1]] - cost_f[x][g[x][0]])/df_sup[x])*cost_f[x][g[x][0]]  if len(g[x]) > 1 else (cost_f[x][g[x][0]]/df_sup[x])*cost_f[x][g[x][0]]

        f = max(d, key=lambda n: d[n])
        t = max(s, key=lambda n: s[n])


        t, f = (f, g[f][0]) if d[f] >= s[t] else (g[t][0], t)


        v = min(df_sup[f], df_dem[t])
        # print(f,t)
        # print(v)
        res[f][t] += v
        df_dem[t] -= v

        if df_dem[t] == 0:
            for k, n in df_sup.items():
                if n != 0:
                    g[k].remove(t)
            del g[t]
            del df_dem[t]
        df_sup[f] -= v
        if df_sup[f] == 0:
            for k, n in df_dem.items():
                if n != 0:
                    g[k].remove(f)
            del g[f]
            del df_sup[f]
    
    cost = 0
    cols = sorted(df_demand.keys())
    for g in sorted(costs1):
        for n in cols:
            y = res[g][n]
            if y != 0:
                pass
            cost += y * costs1[g][n]
            # print ("  ",)
    print("Total Cost yielded by WUPCM2 = ", cost)

In [ ]:
def RCSPCM(costs,supply,demand):
    
    cost_f = copy.deepcopy(costs)
    df_sup =copy.deepcopy(supply)
    df_dem =copy.deepcopy(demand)
    
    res = dict((k, defaultdict(int)) for k in cost_f)
    g = {}
    for x in df_sup:
        g[x] = sorted(cost_f[x].keys(), key=lambda g: cost_f[x][g])
    for x in df_dem:
        g[x] = sorted(cost_f.keys(), key=lambda g: cost_f[g][x])

    flag=0
    while g:
        d = {}
        s = {}
        for x in df_sup:
            s[x] = 0
            for y in df_dem:
                s[x]=s[x] + cost_f[x][y]- cost_f[x][g[x][0]]

        for y in df_dem :
            s[y]=0
            for x in df_sup :
                s[y] =s[y] + cost_f[x][y]- cost_f[g[y][0]][y]
        t,f="S","D"
        rx = max(s, key=lambda n: s[n])
        if df_dem.get(rx)!=None :
            t,f = g[rx][0] , rx
        else :
            t,f = rx , g[rx][0]

        v= min(df_sup[t],df_dem[f])
        df_dem[f]-=v
        df_sup[t]-=v
        res[t][f]+=v

        if df_dem[f]==0:
            for k, n in df_sup.items():
                if n != 0:
                    g[k].remove(f)
            del g[f]
            del df_dem[f]
        if df_sup[t]==0:
            for k, n in df_dem.items():
                if n != 0:
                    g[k].remove(t)
            del g[t]
            del df_sup[t]

    cost = 0
    cols = sorted(df_demand.keys())
    for g in sorted(costs1):
        for n in cols:
            y = res[g][n]
            if y != 0:
                pass
            cost += y * costs1[g][n]
         
    print("Total Cost yielded by RCSPCM = ", cost)

In [ ]:
def MGMM(costs,supply,demand):
    
    cost_f = copy.deepcopy(costs)
    df_sup =copy.deepcopy(supply)
    df_dem =copy.deepcopy(demand)
    
    res = dict((k, defaultdict(int)) for k in cost_f)
    ls=len(df_sup)
    
    while df_sup and df_dem:
        sd={}
        for x in df_dem:
            l=[]
            for y in df_sup:
                l.append(cost_f[y][x])
            if len(df_sup)>1:
                sd[x]=stats.gmean(l)
            else:
                sd[x]=(list(df_sup.values()))[0]

        for y in df_sup :
            l=[]
            for x in df_dem:
                l.append(cost_f[y][x])
            if len(df_dem)>1:
                sd[y]=stats.gmean(l)
            else:
                sd[y]=(list(df_dem.values()))[0]
           
        ma=max(sd.values())
        l=[]
        for ke in sd:
            if sd[ke]==ma:
                l.append(ke)
        ind=l[0]
        
        if df_dem.get(ind)!=None:
            mi=10000
            mind="S"
            for y in df_sup:
                if cost_f[y][ind]<mi:
                    mi=cost_f[y][ind]
                    mind=y
            v=min(df_sup[mind],df_dem[ind])
            df_sup[mind]-=v
            df_dem[ind]-=v
            res[mind][ind]+=v
            if df_sup[mind]==0:
                del df_sup[mind]
            if df_dem[ind]==0:
                del df_dem[ind]
        else:
            mi=10000
            mind="S"
            for y in df_dem:
                if cost_f[ind][y]<mi:
                    mi=cost_f[ind][y]
                    mind=y
            v=min(df_sup[ind],df_dem[mind])
            df_sup[ind]-=v
            df_dem[mind]-=v
            res[ind][mind]+=v
            if df_sup[ind]==0:
                del df_sup[ind]
            if df_dem[mind]==0:
                del df_dem[mind]


    cost = 0
    cols = sorted(df_demand.keys())
    for g in sorted(costs1):
        for n in cols:
            y = res[g][n]
            if y != 0:
                pass
            cost += y * costs1[g][n]
            
    print ("Total Cost yielded by MGMM = ", cost)

In [ ]:
### Self Proposed Heuristic Methods

In [ ]:
def WAMM(costs,supply,demand):
    
    cost_f = copy.deepcopy(costs)
    df_sup =copy.deepcopy(supply)
    df_dem =copy.deepcopy(demand)
    
    res = dict((k, defaultdict(int)) for k in cost_f)
    ls=len(df_sup)
    
    while df_sup and df_dem:
        sd={}
        for x in df_dem:
            l=[]
            for y in df_sup:
                l.append(cost_f[y][x])
            if len(df_sup)>1:
                sd[x]=mean(l)
            else:
                sd[x]=(list(df_sup.values()))[0]

        for y in df_sup :
            l=[]
            for x in df_dem:
                l.append(cost_f[y][x])
            if len(df_dem)>1:
                sd[y]=mean(l)
            else:
                sd[y]=(list(df_dem.values()))[0]
           
        ma=max(sd.values())
        l=[]
        for ke in sd:
            if sd[ke]==ma:
                l.append(ke)
        ind=l[0]
        
        if df_dem.get(ind)!=None:
            mi=10000
            mind="S"
            for y in df_sup:
                if cost_f[y][ind]<mi:
                    mi=cost_f[y][ind]
                    mind=y
            v=min(df_sup[mind],df_dem[ind])
            df_sup[mind]-=v
            df_dem[ind]-=v
            res[mind][ind]+=v
            if df_sup[mind]==0:
                del df_sup[mind]
            if df_dem[ind]==0:
                del df_dem[ind]
        else:
            mi=10000
            mind="S"
            for y in df_dem:
                if cost_f[ind][y]<mi:
                    mi=cost_f[ind][y]
                    mind=y
            v=min(df_sup[ind],df_dem[mind])
            df_sup[ind]-=v
            df_dem[mind]-=v
            res[ind][mind]+=v
            if df_sup[ind]==0:
                del df_sup[ind]
            if df_dem[mind]==0:
                del df_dem[mind]


    cost = 0
    cols = sorted(df_demand.keys())
    for g in sorted(costs1):
        for n in cols:
            y = res[g][n]
            if y != 0:
                pass
            cost += y * costs1[g][n]
            
    print ("Total Cost yielded by WAMM = ", cost)

In [ ]:
def WGMM(costs,supply,demand):
    
    cost_f = copy.deepcopy(costs)
    df_sup =copy.deepcopy(supply)
    df_dem =copy.deepcopy(demand)
    
    res = dict((k, defaultdict(int)) for k in cost_f)
    ls=len(df_sup)
    
    while df_sup and df_dem:
        sd={}
        for x in df_dem:
            l=[]
            for y in df_sup:
                l.append(cost_f[y][x])
            if len(df_sup)>1:
                sd[x]=stats.gmean(l)
            else:
                sd[x]=(list(df_sup.values()))[0]

        for y in df_sup :
            l=[]
            for x in df_dem:
                l.append(cost_f[y][x])
            if len(df_dem)>1:
                sd[y]=stats.gmean(l)
            else:
                sd[y]=(list(df_dem.values()))[0]
           
        ma=max(sd.values())
        l=[]
        for ke in sd:
            if sd[ke]==ma:
                l.append(ke)
        ind=l[0]
        
        if df_dem.get(ind)!=None:
            mi=10000
            mind="S"
            for y in df_sup:
                if cost_f[y][ind]<mi:
                    mi=cost_f[y][ind]
                    mind=y
            v=min(df_sup[mind],df_dem[ind])
            df_sup[mind]-=v
            df_dem[ind]-=v
            res[mind][ind]+=v
            if df_sup[mind]==0:
                del df_sup[mind]
            if df_dem[ind]==0:
                del df_dem[ind]
        else:
            mi=10000
            mind="S"
            for y in df_dem:
                if cost_f[ind][y]<mi:
                    mi=cost_f[ind][y]
                    mind=y
            v=min(df_sup[ind],df_dem[mind])
            df_sup[ind]-=v
            df_dem[mind]-=v
            res[ind][mind]+=v
            if df_sup[ind]==0:
                del df_sup[ind]
            if df_dem[mind]==0:
                del df_dem[mind]


    cost = 0
    cols = sorted(df_demand.keys())
    for g in sorted(costs1):
        for n in cols:
            y = res[g][n]
            if y != 0:
                pass
            cost += y * costs1[g][n]
            
    print ("Total Cost yielded by WGMM = ", cost)

In [ ]:
def WSDM(costs,supply,demand):
    
    cost_f = copy.deepcopy(costs)
    df_sup =copy.deepcopy(supply)
    df_dem =copy.deepcopy(demand)
    
    res = dict((k, defaultdict(int)) for k in cost_f)
    ls=len(df_sup)

    while df_sup and df_dem:
        sd={}
        # print(demand,supply)
        for x in df_dem:
            # print(x)
            l=[]
            # print(len(supply))
            for y in df_sup:
                l.append(cost_f[y][x])
            # print(l)
            if len(df_sup)>1:

                sd[x]=np.std(l)
            else:
                sd[x]=(list(df_sup.values()))[0]
                
        for y in df_sup :
            l=[]
            for x in df_dem:
                l.append(cost_f[y][x])
            if len(df_dem)>1:

                sd[y]=np.std(l)
            else:
                sd[y]=(list(df_dem.values()))[0]
            # sd[y]=stdev(l)
        # print(sd)
        ma=max(sd.values())
        l=[]
        for ke in sd:
            if sd[ke]==ma:
                l.append(ke)
        ind=l[0]
        
        if df_dem.get(ind)!=None:
            mi=10000
            mind="S"
            for y in df_sup:
                if cost_f[y][ind]<mi:
                    mi=cost_f[y][ind]
                    mind=y
            v=min(df_sup[mind],df_dem[ind])
            df_sup[mind]-=v
            df_dem[ind]-=v
            res[mind][ind]+=v
            if df_sup[mind]==0:
                del df_sup[mind]
            if df_dem[ind]==0:
                del df_dem[ind]
        else:
            mi=10000
            mind="S"
            # print(ind)
            for y in df_dem:
                # print(ind,y)
                if cost_f[ind][y]<mi:
                    mi=cost_f[ind][y]
                    mind=y
            v=min(df_sup[ind],df_dem[mind])
            df_sup[ind]-=v
            df_dem[mind]-=v
            res[ind][mind]+=v
            if df_sup[ind]==0:
                del df_sup[ind]
            if df_dem[mind]==0:
                del df_dem[mind]

        # break


    # print(demand,supply)

    cost = 0
    cols = sorted(df_demand.keys())
    for g in sorted(costs1):
        # print (g, "\t",)
        for n in cols:
            y = res[g][n]
            if y != 0:
                pass
                # print (y,)
            cost += y * costs1[g][n]
            # print ("\t",)
        # print()
    print ("Total Cost yielded by WSDM = ", cost)

In [ ]:
def WHMM(costs,supply,demand):
    
    cost_f = copy.deepcopy(costs)
    df_sup =copy.deepcopy(supply)
    df_dem =copy.deepcopy(demand)
    
    res = dict((k, defaultdict(int)) for k in cost_f)
    ls=len(df_sup)

    while df_sup and df_dem:
        sd={}
        # print(demand,supply)
        for x in df_dem:
            # print(x)
            l=[]
            # print(len(supply))
            for y in df_sup:
                l.append(cost_f[y][x])
            # print(l)
            if len(df_sup)>1:

                sd[x]=harmonic_mean(l)
            else:
                sd[x]=(list(df_sup.values()))[0]
                
        for y in df_sup :
            l=[]
            for x in df_dem:
                l.append(cost_f[y][x])
            if len(df_dem)>1:

                sd[y]=harmonic_mean(l)
            else:
                sd[y]=(list(df_dem.values()))[0]
            # sd[y]=stdev(l)
        # print(sd)
        ma=max(sd.values())
        l=[]
        for ke in sd:
            if sd[ke]==ma:
                l.append(ke)
        ind=l[0]
        
        if df_dem.get(ind)!=None:
            mi=10000
            mind="S"
            for y in df_sup:
                if cost_f[y][ind]<mi:
                    mi=cost_f[y][ind]
                    mind=y
            v=min(df_sup[mind],df_dem[ind])
            df_sup[mind]-=v
            df_dem[ind]-=v
            res[mind][ind]+=v
            if df_sup[mind]==0:
                del df_sup[mind]
            if df_dem[ind]==0:
                del df_dem[ind]
        else:
            mi=10000
            mind="S"
            # print(ind)
            for y in df_dem:
                # print(ind,y)
                if cost_f[ind][y]<mi:
                    mi=cost_f[ind][y]
                    mind=y
            v=min(df_sup[ind],df_dem[mind])
            df_sup[ind]-=v
            df_dem[mind]-=v
            res[ind][mind]+=v
            if df_sup[ind]==0:
                del df_sup[ind]
            if df_dem[mind]==0:
                del df_dem[mind]

        # break


    # print(demand,supply)

    cost = 0
    cols = sorted(df_demand.keys())
    for g in sorted(costs1):
        # print (g, "\t",)
        for n in cols:
            y = res[g][n]
            if y != 0:
                pass
                # print (y,)
            cost += y * costs1[g][n]
            # print ("\t",)
        # print()
    print ("Total Cost yielded by WHMM = ", cost)

In [5]:
costs_t, df_supply, df_demand, costs1, df_tdkom = generate_TCM()


In [ ]:
### SSM - TCM
SSM(costs_t,df_supply,df_demand)

In [7]:
### SSM - TOCM
SSM(costs_tocm,df_supply,df_demand)

NameError: name 'SSM' is not defined

In [ ]:
### BCE - TCM
BCE(costs_t,df_supply,df_demand)

In [ ]:
### BCE - TOCM
BCE(costs_tocm,df_supply,df_demand)

In [ ]:
### JHM - TCM
JHM(costs_t,df_supply,df_demand)

In [ ]:
### JHM - TOCM
JHM(costs_tocm,df_supply,df_demand)

In [ ]:
### RAM - TCM
RAM(costs_t,df_supply,df_demand)

In [ ]:
### RAM - TOCM
RAM(costs_tocm,df_supply,df_demand)

In [ ]:
### KSM - TCM
KSM(costs_t,df_supply,df_demand)

In [ ]:
### KSM - TOCM
KSM(costs_tocm,df_supply,df_demand)

In [ ]:
### MT - TCM
MT(costs_t,df_supply,df_demand)

In [ ]:
### MT - TOCM
MT(costs_tocm,df_supply,df_demand)

In [ ]:
### EDM - TCM
EDM(costs_t,df_supply,df_demand)

In [ ]:
### EDM - TOCM
EDM(costs_tocm,df_supply,df_demand)

In [ ]:
### GMM - TCM
GMM(costs_t,df_supply,df_demand)

In [ ]:
### GMM - TOCM
GMM(costs_tocm,df_supply,df_demand)

In [ ]:
### WHMM - TCM
WHMM(costs_t,df_supply,df_demand)

In [ ]:
### WHMM - TOCM
WHMM(costs_tocm,df_supply,df_demand)

In [ ]:
### VAM - TCM
VAM(costs_t,df_supply,df_demand)

In [ ]:
### VAM - TOCM
VAM(costs_tocm,df_supply,df_demand)

In [ ]:
### WSDM - TCM
WSDM(costs_t,df_supply,df_demand)

In [ ]:
### WSDM - TOCM
WSDM(costs_tocm,df_supply,df_demand)

In [ ]:
### WGMM - TCM
WGMM(costs_t,df_supply,df_demand)

In [ ]:
### WGMM - TOCM
WGMM(costs_tocm,df_supply,df_demand)

In [ ]:
### WAMM - TCM
WAMM(costs_t,df_supply,df_demand)

In [ ]:
### WAMM - TOCM
WAMM(costs_tocm,df_supply,df_demand)

In [ ]:
### RCMCAM - TCM
RCMCAM(costs_t,df_supply,df_demand)

In [ ]:
### RCMCAM - TOCM
RCMCAM(costs_tocm,df_supply,df_demand)

In [ ]:
### RCWMCAM - TCM
RCWMCAM(costs_t,df_supply,df_demand)

In [ ]:
### RCWMCAM - TOCM
RCWMCAM(costs_tocm,df_supply,df_demand)

In [11]:
### WPCM1 - TCM
WPCM1(costs_t,df_supply,df_demand)

Total Cost yielded by WPCM1 =  241467.62166493476

In [ ]:
###WPCM2, UPCM, SDM done!

In [ ]:
### WUPCM1 - TCM
WUPCM1(costs_t,df_supply,df_demand)

In [ ]:
### WUPCM1 - TOCM
WUPCM1(costs_tocm,df_supply,df_demand)

In [ ]:
### WUPCM2 - TCM
WUPCM2(costs_t,df_supply,df_demand)

In [ ]:
### WUPCM2- TOCM
WUPCM2(costs_tocm,df_supply,df_demand)

In [ ]:
### RCSPCM - TCM
RCSPCM(costs_t,df_supply,df_demand)

In [ ]:
### RCSPCM- TOCM
RCSPCM(costs_tocm,df_supply,df_demand)

In [ ]:
### MGMM - TCM
MGMM(costs_t,df_supply,df_demand)

In [ ]:
### MGMM- TOCM
MGMM(costs_tocm,df_supply,df_demand)

In [6]:
def WPCM2(costs_tc, df_sup, df_dem):
    
    costs = copy.deepcopy(costs_tc)
    supply = copy.deepcopy(df_sup)
    demand = copy.deepcopy(df_dem)
    cols = sorted(demand.keys())
    
    costs1=copy.deepcopy(costs)
    mi = 10000
    res = dict((k, defaultdict(int)) for k in costs)
    g = {}
    for x in supply:
        g[x] = sorted(costs[x].keys(), key=lambda g: costs[x][g])
    for x in demand:
        g[x] = sorted(costs.keys(), key=lambda g: costs[g][x])
    # print(costs1)
    flag=0
    while g:
        
        d = {}
        # print(demand,supply)

        for x in demand:
            d[x] = (costs[g[x][1]][x] - costs[g[x][0]][x])* costs[g[x][0]][x] if len(g[x]) > 1 else (costs[g[x][0]][x])*costs[g[x][0]][x]
        s = {}
        for x in supply:
            d[x] = (costs[x][g[x][1]] - costs[x][g[x][0]])* costs[x][g[x][0]] if len(g[x]) > 1 else costs[x][g[x][0]]*costs[x][g[x][0]]
        
        for x in supply :
            for y in demand:
                if costs[x][y]*min(supply[x],demand[y])<mi :
                    mi= costs[x][y]*min(supply[x],demand[y])
        
        sd = max(d, key=lambda n: d[n])
        # t = max(s, key=lambda n: s[n])
        t,f="D","S"
        if demand.get(sd)!=None:
            t,f = sd, g[sd][0]
        else:
            t,f = g[sd][0],sd

        
        

        # t,f=tfin,fin
        # print(t,f)
        v = min(supply[f], demand[t])
        res[f][t] += v
        demand[t] -= v
        
        if demand[t] == 0:
            for k, n in supply.items():
                if n != 0:
                    g[k].remove(t)
            del g[t]
            del demand[t]
        supply[f] -= v
        if supply[f] == 0:
            for k, n in demand.items():
                if n != 0:
                    g[k].remove(f)
            del g[f]
            del supply[f]
        
    
    # print(costs1)
    cost = 0
    for g in sorted(costs1):
        # print (g, " ",)
        for n in cols:
            y = res[g][n]
            if y != 0:
                pass
                # print (y,)
            cost += y * costs1[g][n]
            # print ("  ",)
        # print(" ")
    print ("Total Cost = ", cost)

WPCM2(costs_t, df_supply, df_demand)

Total Cost =  240361.36164888885

In [8]:
def SDM(df_tdkom, supply, demand):
    import numpy as np
    df_result = pd.DataFrame(np.zeros(df_tdkom.shape))

    while(list(supply.values()) and list(demand.values())): 
        
        # To find the minimum cost row-wise
        df_nr_list = list(df_tdkom.min(axis=1))
        df_nr = {}
        for i in range(len(df_tdkom.index)):
            df_nr[df_tdkom.index[i]] = df_nr_list[i]
        #df_nr

        # To find the minimum cost column-wise
        df_nc_list = list(df_tdkom.min(axis=0))
        df_nc = {}
        for i in range(len(df_tdkom.columns)):
            df_nc[df_tdkom.columns[i]] = df_nc_list[i]
        #df_nc

        import numpy as np
        stdr = {}
        for i in range(len(df_tdkom.index)):
            stdr[df_tdkom.index[i]] = list(np.std(df_tdkom, axis = 1))[i]
        #stdr

        stdc = {}
        for i in range(len(df_tdkom.columns)):
            stdc[df_tdkom.columns[i]] = list(np.std(df_tdkom, axis = 0))[i]
        #stdc

        r = max(list(stdr.values()) + list(stdc.values()))
        #r

        lst = []
        if(r in stdc.values() and r not in stdr.values()):
            for i in df_tdkom.columns:
                if (stdc[i] == r): 
                    for j in (df_tdkom.index):
                        if (df_tdkom.loc[j,i] == df_nc[i]):
                            lst.append([j,i])
                            
            if (len(lst) == 1):
                df_result.loc[lst[0][0],lst[0][1]] = min(supply[lst[0][0]],demand[lst[0][1]])
                supply[lst[0][0]] = supply[lst[0][0]] - df_result.loc[lst[0][0],lst[0][1]]
                demand[lst[0][1]] = demand[lst[0][1]] - df_result.loc[lst[0][0],lst[0][1]]


                if (supply[lst[0][0]] == 0 and demand[lst[0][1]] == 0):
                    df_tdkom = df_tdkom.drop(lst[0][0], axis = 0)
                    del supply[lst[0][0]]

                elif (supply[lst[0][0]] == 0 and demand[lst[0][1]] != 0):
                    df_tdkom = df_tdkom.drop(lst[0][0], axis = 0)
                    del supply[lst[0][0]]

                elif (supply[lst[0][0]] != 0 and demand[lst[0][1]] == 0):
                    df_tdkom = df_tdkom.drop(lst[0][1], axis = 1)
                    del demand[lst[0][1]]
            
            elif (len(lst)!=1):
                minc = df_tdkom.loc[lst[0][0],lst[0][1]]
                for i in range(len(lst)):
                    if (minc <= df_tdkom.loc[lst[i][0],lst[i][1]]):
                        minc = minc
                    elif (minc > df_tdkom.loc[lst[i][0],lst[i][1]]):
                        minc = df_tdkom.loc[lst[i][0],lst[i][1]]
                        
                lstn = []
                for i in range(len(lst)):
                    if (df_tdkom.loc[lst[i][0],lst[i][1]] == minc):
                        lstn.append([lst[i][0],lst[i][1]])

                df_result.loc[lstn[0][0],lstn[0][1]] = min(supply[lstn[0][0]],demand[lstn[0][1]])
                supply[lstn[0][0]] = supply[lstn[0][0]] - df_result.loc[lstn[0][0],lstn[0][1]]
                demand[lstn[0][1]] = demand[lstn[0][1]] - df_result.loc[lstn[0][0],lstn[0][1]]
                
                if (supply[lstn[0][0]] == 0 and demand[lstn[0][1]] == 0):
                    df_tdkom = df_tdkom.drop(lstn[0][0], axis = 0)
                    del supply[lstn[0][0]]

                elif (supply[lstn[0][0]] == 0 and demand[lstn[0][1]] != 0):
                    df_tdkom = df_tdkom.drop(lstn[0][0], axis = 0)
                    del supply[lstn[0][0]]

                elif (supply[lstn[0][0]] != 0 and demand[lstn[0][1]] == 0):
                    df_tdkom = df_tdkom.drop(lstn[0][1], axis = 1)
                    del demand[lstn[0][1]]


        lstm = [] 
        lstr = [] 
        #### Tie in value of SD, selecting minimum TC
        ### Only if same SD appears row-wise and once column-wise
        if(r in stdc.values() and r in stdr.values()):
            min_val = []
            idx_list = []
            for i in df_tdkom.index:
                for j in (df_tdkom.columns):
                    if (stdr[i] == r and stdc[j] == r):
                        if (df_nr[i] <= df_nc[j]):
                            for l in df_tdkom.columns:
                                if (df_tdkom.loc[i,l] == df_nr[i]):
                                    #print(df_tdkom.loc[i,l], df_nr[i])
                                    min_val.append(df_nr[i])
                                    idx_list.append([i,l])
                                    #lstm.append([i,l])
                        elif (df_nr[i] > df_nc[j]):
                            for o in df_tdkom.index:
                                if (df_tdkom.loc[o,j] == df_nc[j]):
                                        min_val.append(df_nc[j])
                                        idx_list.append([o,j])
                                        #lstm.append([o,j]) 
                        lstm.append(idx_list[min_val.index(min(min_val))])         
    

            ###### Allocating to the left most cell, since that' how it identifies
            df_result.loc[lstm[0][0],lstm[0][1]] = min(supply[lstm[0][0]],demand[lstm[0][1]])
            supply[lstm[0][0]] = supply[lstm[0][0]] - df_result.loc[lstm[0][0],lstm[0][1]]
            demand[lstm[0][1]] = demand[lstm[0][1]] - df_result.loc[lstm[0][0],lstm[0][1]]
            
            
            if (supply[lstm[0][0]] == 0 and demand[lstm[0][1]] == 0):
                #print(supply[lstm[0][0]], demand[lstm[0][1]] )
                df_tdkom = df_tdkom.drop(lstm[0][0], axis = 0)
                del supply[lstm[0][0]]
                
            elif (supply[lstm[0][0]] == 0 and demand[lstm[0][1]] != 0):
                #print(supply[lstm[0][0]], demand[lstm[0][1]] )
                df_tdkom = df_tdkom.drop(lstm[0][0], axis = 0)
                del supply[lstm[0][0]]
            
            elif (supply[lstm[0][0]] != 0 and demand[lstm[0][1]] == 0):
                #print(supply[lstm[0][0]], demand[lstm[0][1]] )
                df_tdkom = df_tdkom.drop(lstm[0][1], axis = 1)
                del demand[lstm[0][1]]

    
        if (r in stdr.values() and r not in stdc.values()):
            for i in df_tdkom.index:
                if (stdr[i] == r): 
                    for j in (df_tdkom.columns):
                        if (df_tdkom.loc[i,j] == df_nr[i]):
                            lstr.append([i,j])
            
            if (len(lstr)==1):
                df_result.loc[lstr[0][0],lstr[0][1]] = min(supply[lstr[0][0]],demand[lstr[0][1]])
                supply[lstr[0][0]] = supply[lstr[0][0]] - df_result.loc[lstr[0][0],lstr[0][1]]
                demand[lstr[0][1]] = demand[lstr[0][1]] - df_result.loc[lstr[0][0],lstr[0][1]]  

                if (supply[lstr[0][0]] == 0 and demand[lstr[0][1]] == 0):
                    df_tdkom = df_tdkom.drop(lstr[0][0], axis = 0)
                    del supply[lstr[0][0]]

                elif (supply[lstr[0][0]] == 0 and demand[lstr[0][1]] != 0):
                    df_tdkom = df_tdkom.drop(lstr[0][0], axis = 0)
                    del supply[lstr[0][0]]

                elif (supply[lstr[0][0]] != 0 and demand[lstr[0][1]] == 0):
                    df_tdkom = df_tdkom.drop(lstr[0][1], axis = 1)
                    del demand[lstr[0][1]]
                    
            elif (len(lstr)!=1):
                minr = df_tdkom.loc[lstr[0][0],lstr[0][1]]
                for i in range(len(lstr)):
                    if (minr <= df_tdkom.loc[lstr[i][0],lstr[i][1]]):
                        minr = minr
                    elif (minr > df_tdkom.loc[lstr[i][0],lstr[i][1]]):
                        minr = df_tdkom.loc[lstr[i][0],lstr[i][1]]

                lstp = []
                for i in range(len(lstr)):
                    if (df_tdkom.loc[lstr[i][0],lstr[i][1]] == minr):
                        lstp.append([lstr[i][0],lstr[i][1]])

                df_result.loc[lstp[0][0],lstp[0][1]] = min(supply[lstp[0][0]],demand[lstp[0][1]])
                supply[lstp[0][0]] = supply[lstp[0][0]] - df_result.loc[lstp[0][0],lstp[0][1]]
                demand[lstp[0][1]] = demand[lstp[0][1]] - df_result.loc[lstp[0][0],lstp[0][1]]

                if (supply[lstp[0][0]] == 0 and demand[lstp[0][1]] == 0):
                    df_tdkom = df_tdkom.drop(lstp[0][0], axis = 0)
                    del supply[lstp[0][0]]

                elif (supply[lstp[0][0]] == 0 and demand[lstp[0][1]] != 0):
                    df_tdkom = df_tdkom.drop(lstp[0][0], axis = 0)
                    del supply[lstp[0][0]]

                elif (supply[lstp[0][0]] != 0 and demand[lstp[0][1]] == 0):
                    df_tdkom = df_tdkom.drop(lstp[0][1], axis = 1)
                    del demand[lstp[0][1]]
    print ("Total Cost = ", df_result)

In [ ]:
SDM(df_tdkom, df_supply, df_demand)

In [8]:
UPCM(costs_t, df_supply, df_demand)

Total Cost =  238182.76354743564